## Setup

In [2]:
# Om te importen moet je een terminal openen (vs code) en de volende code invullen: py -m pip install numpy
import pandas as pd
import numpy as np
import matplotlib as plt
import re

pd.set_option("display.max.columns", None) #Make sure all columns are showed


ModuleNotFoundError: No module named 'pandas'

## Import data

In [26]:
df_train = pd.read_csv("https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/test.csv")
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  891 non-null    int64  
 1   survived     891 non-null    int64  
 2   pclass       891 non-null    int64  
 3   name         891 non-null    object 
 4   sex          891 non-null    object 
 5   age          714 non-null    float64
 6   sibsp        891 non-null    int64  
 7   parch        891 non-null    int64  
 8   ticket       891 non-null    object 
 9   fare         891 non-null    float64
 10  cabin        204 non-null    object 
 11  embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  418 non-null    int64  
 1   pcl

## Check Missings

In [27]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())


passengerid      0
survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         2
dtype: int64
passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             1
cabin          327
embarked         0
dtype: int64


Cabin en Age hebben veel missings -> Hier moeten we iets mee

Missings bij embarked en fare verwijderen we voor het gemak.

In [28]:
df_train = df_train.dropna(subset=['embarked'])
df_test = df_test.dropna(subset=['fare'])

print(df_train.isnull().sum())
print(df_test.isnull().sum())

passengerid      0
survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         0
dtype: int64
passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             0
cabin          326
embarked         0
dtype: int64


## Restructure data

In [35]:
print(df_train.apply(lambda col: col.unique()))

# Verschillende kollomen bevatten nan. Dit wordt niet gelezen als missing dus passen we die aan naar NaN
df_train = df_train.replace("nan", "NaN")


passengerid     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
survived                                                   [0, 1]
pclass                                                  [3, 1, 2]
name            [Braund, Mr. Owen Harris, Cumings, Mrs. John B...
sex                                                [male, female]
age             [22.0, 38.0, 26.0, 35.0, nan, 54.0, 2.0, 27.0,...
sibsp                                       [1, 0, 3, 4, 2, 5, 8]
parch                                       [0, 1, 2, 5, 3, 4, 6]
ticket          [A/5 21171, PC 17599, STON/O2. 3101282, 113803...
fare            [7.25, 71.2833, 7.925, 53.1, 8.05, 8.4583, 51....
cabin           [nan, C85, C123, E46, G6, C103, D56, A6, C23 C...
embarked                                                [S, C, Q]
sex_cat                                                    [1, 0]
pclass_cat                                              [2, 0, 1]
embarked_cat                                            [2, 0, 1]
dtype: obj

In [31]:
# Categoriseren
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cat_list =  ["sex", "pclass", "embarked"]

for x in cat_list:
  df_train[x + '_cat'] = le.fit_transform(df_train[x])
  print(list(le.classes_))
  

['female', 'male']
[1, 2, 3]
['C', 'Q', 'S']


In [73]:
# Groeperen:
## Age -> agegroups
df_train.age.describe()

bins = [0, 24.999, 74.999, 100]
group_names = [1,2,3]
df_train['age_group'] = pd.cut(df_train['age'], bins, labels=group_names)



In [ ]:
# Check kinderen onder 18 en relatie tot ouders

ft = df_train.sex_cat.value_counts()
pd.crosstab(index=df_train['survived'], columns=df_train['sex_cat'],
 dropna=False, normalize=False, margins = True, margins_name= "Total")

dt = df_train.sex_cat.dtype

In [90]:
# Exract titel uit naam:
print(df_train["name"])
df_train["title"] = df_train["name"].str.extract('((?<=,)\s[a-zA-Z]*)')
df_train["title"] = df_train["title"].str.split()

df_train['title'] = df_train['title'].str.replace('Ms', 'Miss')

df_train.title.value_counts()

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: name, Length: 889, dtype: object


Series([], Name: title_2, dtype: int64)